In [1]:
import numpy as np
import cv2 as cv
import napari

In [2]:
def visualize_napari(images, labels):
    """ Visualize all images in list using napari
    parameters
    ----------
    imagest: list of numpy images
    labels: list of names for each image
        """
    viewer = napari.Viewer()
    for i, img in enumerate(images):
        viewer.add_image(img, name=labels[i])

In [3]:
# img = cv.imread('DE_annotated_pictures/20220202_547 D4 001_ch00_yn.tif')
img = cv.imread('DE_annotated_pictures/20220918_812 100ml D4001_ch00_yn.tif')
# viewer = napari.Viewer()
# viewer.add_image(img, rgb=True)

In [223]:
img1 = cv.imread('DE_annotated_pictures/20220528_703 D4 001_ch00_yn.tif')
img2 = cv.imread('DE_annotated_pictures/20220918_811 D4001_ch00_yn.tif')
print(img1.shape, img2.shape)
viewer = napari.Viewer()
viewer.add_image(img1)
viewer.add_image(img2)

(1944, 2592, 3) (1944, 2592, 3)


<Image layer 'img2' at 0x2ed6440d0c0>

In [4]:
gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
# viewer.add_image(gray_img)

In [34]:
low = np.array([0, 0, 0])
high = np.array([200, 180, 105])
mask_inRange = cv.inRange(img, low, high)
masked_img = cv.bitwise_and(img, img, mask=mask_inRange)
# viewer.add_image(mask_inRange)

In [24]:
twoDimage = img.reshape((-1,3))
twoDimage = np.float32(twoDimage)

In [25]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
attempts = 10

ret1, label1, center1 = cv.kmeans(twoDimage, 2, None, criteria, attempts, cv.KMEANS_PP_CENTERS)
ret2, label2, center2 = cv.kmeans(twoDimage, 3, None, criteria, attempts, cv.KMEANS_PP_CENTERS)
ret3, label3, center3 = cv.kmeans(twoDimage, 4, None, criteria, attempts, cv.KMEANS_PP_CENTERS)

center1 = np.uint8(center1)
res1 = center1[label1.flatten()]
mask_2nn = res1.reshape((img.shape))

center2 = np.uint8(center2)
res2 = center2[label2.flatten()]
mask_3nn = res2.reshape((img.shape))

center3 = np.uint8(center3)
res3 = center3[label3.flatten()]
mask_4nn = res3.reshape((img.shape))

# viewer.add_image(mask_2nn)
# viewer.add_image(mask_3nn)
# viewer.add_image(mask_4nn)

In [5]:
_, thresh = cv.threshold(gray_img, np.mean(gray_img), 255, cv.THRESH_BINARY_INV)
edges = cv.dilate(cv.Canny(thresh, 0, 255), None)

In [6]:
kernel1 = cv.getStructuringElement(cv.MORPH_RECT, (3,3))
kernel2 = cv.getStructuringElement(cv.MORPH_ELLIPSE, (4,4))

erosion = cv.erode(thresh, kernel2, iterations = 5)
opening = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel2)
closing = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel2)
dilation = cv.dilate(thresh, kernel2, iterations = 1)

In [248]:
viewer = napari.Viewer()
viewer.add_image(img)
viewer.add_image(thresh)
viewer.add_image(erosion)
# viewer.add_image(opening) 
# viewer.add_image(closing)
# viewer.add_image(dilation)

<Image layer 'erosion' at 0x2ed4b8dbfa0>

In [131]:
thresh_adpt_m = cv.adaptiveThreshold(gray_img, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 25, 10)
thresh_adpt_g = cv.adaptiveThreshold(gray_img, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, -10)
dilation = cv.dilate(thresh_adpt_m, kernel1, iterations = 1)
opening = cv.morphologyEx(thresh_adpt_m, cv.MORPH_OPEN, kernel1)

In [148]:
viewer = napari.Viewer()
viewer.add_image(thresh_adpt_m)
viewer.add_image(dilation)
viewer.add_image(opening)
viewer.add_image(img)

<Image layer 'img' at 0x2ed34103bb0>

In [31]:
# viewer.add_image(th2)
# viewer.add_image(th3)

In [39]:
from skimage.filters import threshold_otsu

th_otsu = threshold_otsu(gray_img)
thresh_otsu  = gray_img < th_otsu

# viewer.add_image(img_otsu)

In [44]:
imgs = [img, gray_img, mask_inRange, thresh, opening, closing, dilation, thresh_adpt_m, thresh_adpt_g, thresh_otsu]
labels = ['original', 'grayscale', 'color masking', 'global thresholding', 
          'opening', 'opened and closed', 'opened and dilated',
          'adaptive mean thresholding', 'adpative gaussian thresholding', 'otsu thresholding']
print(len(imgs), len(labels))
visualize_napari(imgs, labels)

10 10


In [7]:
num_labels, labels, stats, centroids = cv.connectedComponentsWithStats(erosion, connectivity=8)

In [250]:
def clean_stats(labels, stats, centroids):
    """
    Delete connected components that are too small or too large.
    """
    labels_cp = labels.copy()
    stats_cp = stats.copy()
    centroids_cp = centroids.copy()
    indices = []

    for i in range(0, stats.shape[0]):
        if stats[i, 4] < 200:
            indices.append(i)
        elif stats[i, 4] > 300**2:
            indices.append(i)
        elif not(0.7 <= stats[i,2] / stats[i,3] <= 1.3):
            indices.append(i)
    
    labels_cp = labels_cp * ~(np.in1d(labels_cp, indices).reshape(labels_cp.shape))
    stats_cp = np.delete(stats_cp, indices, axis=0)
    centroids_cp = np.delete(centroids_cp, indices, axis=0)
    return labels_cp, stats_cp, centroids_cp

In [251]:
tlabels, tstats, tcentroids = clean_stats(labels, stats, centroids)

In [252]:
tstats.shape

(57, 5)

In [20]:
def add_bounding_boxes(img, stats, labels, centroids):
    """
    Add rectangles around objects
    """
    labels_cp = labels.copy()
    stats_cp = stats.copy()
    centroids_cp = centroids.copy()
    indices = []
    img_copy = img.copy()
    
    for i in range(0, stats.shape[0]):
        # discard component if too small indicating noise or holes within components
        if stats[i, 4] < 200:
            indices.append(i)
            continue
        # color box red if component too big indicating two components being attached to each other
        # or if ratio between width and height of component outside tolerance 
        # indicating two components being attached to each other or component being at boundary and thus cropped
        elif stats[i, 4] > 300**2 or not(0.7 <= stats[i,2] / stats[i,3] <= 1.3):
            indices.append(i)
            x = stats[i, 0] - 10
            y = stats[i, 1] - 10
            w = stats[i, 2] + 20
            h = stats[i, 3] + 20
            cv.rectangle(img_copy, (x, y), (x + w, y + h), (255, 0, 0), 3)
        # discard component if at boundary
        elif np.min(np.where(labels == i)[0]) == 0 or np.min(np.where(labels == i)[1]) == 0 \
            or np.max(np.where(labels == i)[0]) == 1943 or np.max(np.where(labels == i)[1]) == 2591:
            indices.append(i)
            x = stats[i, 0] - 10
            y = stats[i, 1] - 10
            w = stats[i, 2] + 20
            h = stats[i, 3] + 20
            cv.rectangle(img_copy, (x, y), (x + w, y + h), (255, 0, 0), 3)
        # else, color box green
        else:
            x = stats[i, 0] - 10
            y = stats[i, 1] - 10
            w = stats[i, 2] + 20
            h = stats[i, 3] + 20
            cv.rectangle(img_copy, (x, y), (x + w, y + h), (0, 255, 0), 3)
    
    labels_cp = labels_cp * ~(np.in1d(labels_cp, indices).reshape(labels_cp.shape))
    stats_cp = np.delete(stats_cp, indices, axis=0)
    centroids_cp = np.delete(centroids_cp, indices, axis=0)
    return img_copy, labels_cp, stats_cp, centroids_cp

In [19]:
img.shape

(1944, 2592, 3)

In [21]:
img_with_boxes, tlabels, tstats, tcentroids = add_bounding_boxes(img, stats, labels, centroids)

In [22]:
viewer = napari.Viewer()
viewer.add_image(img_with_boxes)
viewer.add_image(erosion)

<Image layer 'erosion' at 0x20ef021ca60>

In [311]:
def find_enclosing_ellipses(img, labels):
    """
    Find and add enclosing ellipses around objects
    ------------
    labels: contains only labels of "valid" components
    """
    img_copy = img.copy()
    for i in np.unique(labels)[1:]:
        points = np.where(labels == i)
        points = np.transpose(points)
        (xc, yc), (d1, d2), angle = cv.fitEllipse(points)
        ellipse = ((int(yc), int(xc)), (int(d1), int(d2)), angle)
        cv.ellipse(img_copy, ellipse, (0, 0, 255), 3)
    return img_copy

In [312]:
img_with_ellipses = find_enclosing_ellipses(img, tlabels)
viewer.add_image(img_with_ellipses)

<Image layer 'img_with_ellipses' at 0x2ec8f380910>